In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    confusion_matrix,
    roc_curve,
)
from tqdm import tqdm
import nibabel as nib
import matplotlib.pyplot as plt
import torchio as tio
from nilearn.masking import compute_brain_mask
from torchvision import models
from sklearn.preprocessing import label_binarize


In [ ]:
# Function to clip background slices in all three axes
def clip_background_slices(image_data, threshold=0):
    non_empty_slices_axial = [i for i in range(image_data.shape[2]) if np.max(image_data[:, :, i]) > threshold]
    clipped_image_axial = image_data[:, :, non_empty_slices_axial]
    non_empty_slices_coronal = [i for i in range(clipped_image_axial.shape[1]) if np.max(clipped_image_axial[:, i, :]) > threshold]
    clipped_image_coronal = clipped_image_axial[:, non_empty_slices_coronal, :]
    non_empty_slices_sagittal = [i for i in range(clipped_image_coronal.shape[0]) if np.max(clipped_image_coronal[i, :, :]) > threshold]
    clipped_image_sagittal = clipped_image_coronal[non_empty_slices_sagittal, :, :]
    return clipped_image_sagittal

# Function to normalize image data
def normalize_image(image_data):
    transform = tio.RescaleIntensity(out_min_max=(0, 1))
    image_data = transform(torch.tensor(image_data).unsqueeze(0)).squeeze(0).numpy()
    return image_data

# Function to visualize a few slices
def visualize_slices(image_data, title, num_slices=5):
    fig, axs = plt.subplots(1, num_slices, figsize=(15, 5))
    print(image_data.shape)
    for i in range(num_slices):
        slice_idx = image_data.shape[2] // (num_slices + 1) * (i + 1)
        axs[i].imshow(image_data[:, :, slice_idx], cmap='gray')
        axs[i].set_title(f'Slice {slice_idx}')
    plt.suptitle(title)
    plt.show()

# Function to preprocess NIfTI file and visualize each step
def preprocess_nifti_with_visualization(file_path, target_size=(224, 224)):
    try:
        # Load NIfTI image
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()
        
        # Convert sagittal view to axial view
        axial_image = np.transpose(image_data, (0, 2, 1))
        #visualize_slices(axial_image, "Axial View of Original Image")

        # Skull stripping
        brain_mask = compute_brain_mask(nifti_image, threshold=0.1).get_fdata()

        # Check if brain mask is empty
        if np.sum(brain_mask) == 0:
            print(f"Skipping {file_path} due to empty brain mask.")
            return None
        
        # Transpose brain mask to match axial image
        brain_mask = np.transpose(brain_mask, (0, 2, 1))
        stripped_image = axial_image * brain_mask
        #visualize_slices(stripped_image, "Skull Stripped Image")

        # Resample to (1, 1, 1)
        resample = tio.Resample((1, 1, 1), image_interpolation='linear')
        subject = tio.Subject(image=tio.ScalarImage(tensor=stripped_image[np.newaxis, ...]))
        resampled_subject = resample(subject)
        image_data_resampled = resampled_subject.image.data.numpy().squeeze()
        #visualize_slices(image_data_resampled, "Resampled Image")

        # Normalize image data
        normalized_image = normalize_image(image_data_resampled)
        #visualize_slices(normalized_image, "Normalized Image")

        # Clip background slices
        image_data_clipped = clip_background_slices(normalized_image)
        #visualize_slices(image_data_clipped, "Clipped Image")

        # Extract 50 slices from the middle
        middle = image_data_clipped.shape[2] // 2
        start = max(0, middle - 25)
        end = min(image_data_clipped.shape[2], middle + 25)
        extracted_slices = image_data_clipped[:, :, start:end]
        #visualize_slices(extracted_slices, "Extracted Middle Slices")

        # Resize slices to target size
        resized_slices = []
        for slice_idx in range(extracted_slices.shape[2]):
            slice_data = extracted_slices[:, :, slice_idx]
            resized_slice = F.interpolate(torch.tensor(slice_data).unsqueeze(0).unsqueeze(0).float(), size=target_size, mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
            resized_slices.append(resized_slice.numpy())  
        resized_slices = np.stack(resized_slices, axis=-1)  
        #visualize_slices(resized_slices, "Resized Slices")

        return resized_slices
    
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

# Load all samples and preprocess
root_dir = '/kaggle/input/adni3dataset/ADNI3'
classes = {'AD': 0, 'CN': 1, 'MCI': 2 , 'EMCI': 3}
samples = []
for class_label, class_idx in classes.items():
    class_dir = os.path.join(root_dir, f'{class_label}_Collection/ADNI')
    for root, _, files in os.walk(class_dir):
        for file_name in files:
            if file_name.endswith('.nii'):
                file_path = os.path.join(root, file_name)
                preprocessed_data = preprocess_nifti_with_visualization(file_path)
                if preprocessed_data is not None:
                    samples.append((preprocessed_data, class_idx))


all_slices = []
for data, class_idx in samples:
    for slice_idx in range(data.shape[2]):
        slice_data = data[:, :, slice_idx]
        all_slices.append((slice_data, class_idx))

In [ ]:
class ExtractedSlicesDataset(Dataset):
    def __init__(self, slices):
        self.slices = slices

    def __len__(self):
        return len(self.slices)

    def __getitem__(self, idx):
        slice_data, class_idx = self.slices[idx]
        
        # Convert slice_data to torch tensor
        slice_data = torch.tensor(slice_data).float()
        
        # Repeat channel dimension to make 3 channels
        resized_slice = slice_data.repeat(3, 1, 1)  # Shape: [3, 224, 224]

        return resized_slice, class_idx

extracted_slices_dataset = ExtractedSlicesDataset(all_slices)

In [ ]:
# Create DataLoader for the full dataset
batch_size = 25
full_loader = DataLoader(extracted_slices_dataset, batch_size=batch_size, shuffle=True)

# Print shapes
print(f"Full dataset size: {len(extracted_slices_dataset)}")

for i, (slices, label) in enumerate(full_loader):
    if i >= 1:  # Visualize only the first batch
        break
    print(f"Batch {i+1}:")
    print("Slices shape:", slices.shape)
    print("Labels:", label)


In [ ]:
# Visualize a few slices
num_slices_to_visualize = 5  
fig, axes = plt.subplots(1, num_slices_to_visualize, figsize=(15, 5))

for i in range(num_slices_to_visualize):
    slice_data, class_idx = extracted_slices_dataset[40+i]
    slice_data_np = slice_data.numpy()  
    
    if slice_data_np.shape[0] == 3:
        slice_data_np = slice_data_np[0]  # Grayscale, take only one channel
    
    axes[i].imshow(slice_data_np, cmap='gray')
    axes[i].set_title(f"Class: {class_idx}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=4):
        super(ModifiedAlexNet, self).__init__()
        
        # Load the pre-trained AlexNet model
        self.alexnet = models.alexnet(pretrained=False)
        
        self.alexnet.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),  
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)  
        )
        
    def forward(self, x):
        return self.alexnet(x)

# Example usage:
model = ModifiedAlexNet(num_classes=4)


In [ ]:
# Hyperparameters
num_epochs = 100
learning_rate = 0.001
betas = (0.9, 0.999)
eps = 1e-07
batch_size = 25
num_classes = 4  
k_folds = 10
checkpoint_dir = '/kaggle/working/'
random_seed = 42  # Fixed random seed for reproducibility

# Create directory for checkpoints if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

# Initialize device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize KFold with a fixed random seed
kf = KFold(n_splits=k_folds, shuffle=True, random_state=random_seed)

# Function to calculate specificity and sensitivity
def specificity_sensitivity(y_true, y_pred, num_classes):
    cm = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))
    specificities = []
    sensitivities = []
    
    for i in range(num_classes):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp
        fp = np.sum(cm[:, i]) - tp
        tn = np.sum(cm) - (tp + fn + fp)

        specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
        
        specificities.append(specificity)
        sensitivities.append(sensitivity)
    
    # Return the average specificity and sensitivity across all classes
    return np.mean(specificities), np.mean(sensitivities)

def run_folds(fold_start, fold_end):
    for fold, (train_ids, test_ids) in enumerate(kf.split(extracted_slices_dataset), start=fold_start):
        if fold >= fold_end:
            break
        
        print(f'Fold {fold + 1}/{k_folds}')
        
        train_sampler = SubsetRandomSampler(train_ids)
        test_sampler = SubsetRandomSampler(test_ids)
        
        # Create loaders
        train_loader = DataLoader(extracted_slices_dataset, batch_size=batch_size, sampler=train_sampler)
        test_loader = DataLoader(extracted_slices_dataset, batch_size=batch_size, sampler=test_sampler)
        
        # Initialize model, optimizer, loss function
        model = ModifiedAlexNet(num_classes=4).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=betas, eps=eps)
        criterion = nn.CrossEntropyLoss()

        best_metrics = {}

        for epoch in range(num_epochs):
            # Training loop
            model.train()
            train_loss_total = 0.0
            train_correct = 0
            train_total = 0

            for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} Training', leave=False):
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()

                train_loss_total += loss.item() * inputs.size(0)

            train_loss = train_loss_total / len(train_loader.dataset)
            train_acc = 100.0 * train_correct / train_total

            # Test loop
            model.eval()
            test_loss_total = 0.0
            test_correct = 0
            test_total = 0
            all_labels = []
            all_predictions = []
            all_probs = []

            with torch.no_grad():
                for inputs, labels in tqdm(test_loader, desc=f'Test Epoch {epoch + 1}', leave=False):
                    inputs, labels = inputs.to(device), labels.to(device)

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, predicted = torch.max(outputs, 1)
                    test_total += labels.size(0)
                    test_correct += (predicted == labels).sum().item()

                    test_loss_total += loss.item() * inputs.size(0)

                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())
                    all_probs.extend(torch.softmax(outputs, dim=1).cpu().numpy())

            test_loss = test_loss_total / len(test_loader.dataset)
            test_acc = 100.0 * test_correct / test_total

            # Compute additional metrics
            f1 = f1_score(all_labels, all_predictions, average='weighted')
            precision = precision_score(all_labels, all_predictions, average='weighted')
            recall = recall_score(all_labels, all_predictions, average='weighted')
            specificity, sensitivity = specificity_sensitivity(all_labels, all_predictions, num_classes=num_classes)

            # Binarize labels for ROC-AUC calculation 
            all_labels_binarized = label_binarize(all_labels, classes=list(range(num_classes)))
            roc_auc = roc_auc_score(all_labels_binarized, np.array(all_probs), multi_class="ovr", average="weighted")

            if train_loss < 0.0001:
                print(f'Train loss is zero at epoch {epoch + 1}, stopping training.')
                break

        # Save the model after training
        torch.save(model.state_dict(), os.path.join(checkpoint_dir, f'fold_{fold + 1}_model.pth'))

        # Print best metrics for the fold after training
        print(f'\nMetrics for Fold {fold + 1}:')
        print(f'Test Accuracy: {test_acc:.2f}%')
        print(f'F1 Score: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}')
        print(f'Specificity: {specificity:.4f} | Sensitivity: {sensitivity:.4f} | ROC-AUC: {roc_auc:.4f}\n')

        # Plot ROC curve for all classes
        plt.figure()
        for i in range(num_classes):
            fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], np.array(all_probs)[:, i])
            plt.plot(fpr, tpr, label=f'Class {i} (area = {roc_auc:.4f})')

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve for Fold {fold + 1}')
        plt.legend(loc='best')
        plt.show()

run_folds(0, k_folds)


